In [16]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("train_master.csv")


,Train No,Train Name,SEQ,Station Code,Station Name,Arrival time,Departure Time,Distance,Source Station,Source Station Name,Destination Station,Destination Station Name
0,107,SWV-MAO-VLNK,1,SWV,SAWANTWADI R,00:00:00,10:25:00,0,SWV,SAWANTWADI ROAD,MAO,MADGOAN JN.
1,107,SWV-MAO-VLNK,2,THVM,THIVIM,11:06:00,11:08:00,32,SWV,SAWANTWADI ROAD,MAO,MADGOAN JN.
2,107,SWV-MAO-VLNK,3,KRMI,KARMALI,11:28:00,11:30:00,49,SWV,SAWANTWADI ROAD,MAO,MADGOAN JN.
3,107,SWV-MAO-VLNK,4,MAO,MADGOAN JN.,12:10:00,00:00:00,78,SWV,SAWANTWADI ROAD,MAO,MADGOAN JN.
4,108,VLNK-MAO-SWV,1,MAO,MADGOAN JN.,00:00:00,20:30:00,0,MAO,MADGOAN JN.,SWV,SAWANTWADI ROAD


In [18]:
df.dtypes

Train No                     int64
Train Name                  object
SEQ                          int64
Station Code                object
Station Name                object
Arrival time                object
Departure Time              object
Distance                     int64
Source Station              object
Source Station Name         object
Destination Station         object
Destination Station Name    object
dtype: object

# Creating a table to store train number and their respective names, along with source and destination stations

In [10]:
table1 = df.filter(["Train No","Train Name","Source Station","Destination Station"])
# dropping duplicate values
table1=table1.drop_duplicates().reset_index(drop = True)
table1

,Train No,Train Name,Source Station,Destination Station
0,107,SWV-MAO-VLNK,SWV,MAO
1,108,VLNK-MAO-SWV,MAO,SWV
2,128,MAO-KOP SPEC,MAO,KOP
3,290,PALACE ON WH,DSJ,DSJ
4,401,BSB BHARATDA,AWB,BSB
...,...,...,...,...
11107,99904,PUNE-TGN EMU,PUNE,TGN
11108,99905,EMU,TGN,SVJR
11109,99906,EMU,PUNE,TGN
11110,99907,EMU,TGN,PUNE


In [14]:
# storing as a csv file
table1.to_csv("train.csv",index=False)

# creating a table to store station code and station name

In [11]:
table2 = df.filter(["Station Code","Station Name"])
# dropping duplicate values
table2=table2.drop_duplicates().reset_index(drop = True)
table2

,Station Code,Station Name
0,SWV,SAWANTWADI R
1,THVM,THIVIM
2,KRMI,KARMALI
3,MAO,MADGOAN JN.
4,KUDL,KUDAL
...,...,...
8143,VDLR,MUMBAI VADAL
8144,BEPR,BELAPUR C.B.
8145,MNKD,MANKHURD
8146,CMBR,CHEMBUR


In [15]:
# storing as csv
table2.to_csv("station.csv",index=False)

# creating a table with halt timings

In [51]:
table3=df.filter(["Train No","Station Code","Arrival time","Departure Time","Source Station","Destination Station"])
halts=[]
for ind,row in table3.iterrows():
    dh,dm,dse = row["Departure Time"].split(":")
    ah,am,ase = row["Arrival time"].split(":")
    hh=0;hm=0;h_str="";m_str=""
    if(int(dh)<int(ah)):
        hh=24-int(ah)+int(dh)
        if(int(am)>int(dm)):
            hm=60-int(am)+int(dm)
            hh=hh-1
        else:
            hm=int(dm)-int(am)
    else:
        hh=int(dh)-int(ah)
        if(int(am)>int(dm)):
            hm=60-int(am)+int(dm)
            hh=hh-1
        else:
            hm=int(dm)-int(am)
    if hh<10:
        h_str='0'+str(hh)
    else:
        h_str=str(hh)
    if hm<10:
        m_str='0'+str(hm)
    else:
        m_str=str(hm)
    halt=h_str+":"+m_str+":00"
    halts.append(halt)
    if row["Station Code"]==row["Destination Station"]:
       row["Departure Time"]==""
    if row["Station Code"]==row["Source Station"]:
        row["Arrival time"]==""
table3["Halt Time"]=halts
table3 = table3.drop(columns = ["Arrival time","Departure Time","Source Station","Destination Station"],axis = 1)
table3

,Train No,Station Code,Halt Time
0,107,SWV,10:25:00
1,107,THVM,00:02:00
2,107,KRMI,00:02:00
3,107,MAO,11:50:00
4,108,MAO,20:30:00
...,...,...,...
186114,99908,AKRD,00:01:00
186115,99908,DEHR,00:01:00
186116,99908,BGWI,00:01:00
186117,99908,GRWD,00:01:00


In [59]:
# storing as csv
table3.to_csv("halts.csv",index=False)


# table of stations where the train arrives at midnight

In [55]:
table4 = df.filter(["Train no","Station Code","Arrival time","Destination Station"])
table4=table4[(table4["Station Code"]==table4["Destination Station"]) & (df["Arrival time"]=="00:00:00")]
# we do not require the columns destination station and arrival time
table4=table4.drop(columns=["Arrival time","Destination Station"])
table4

,Station Code
79483,LKPR
94284,TBM
95791,MSB
155425,UMB


In [56]:
# saving as csv
table4.to_csv("arrive_midnight.csv")

# storing all stops of a train to find path

In [57]:
table5 = df.filter(["Train No","Station Code","SEQ"])
table5

,Train No,Station Code,SEQ
0,107,SWV,1
1,107,THVM,2
2,107,KRMI,3
3,107,MAO,4
4,108,MAO,1
...,...,...,...
186114,99908,AKRD,8
186115,99908,DEHR,9
186116,99908,BGWI,10
186117,99908,GRWD,11


In [60]:
table5.to_csv("stops.csv")